## 50.021 Artificial Intelligence
## Homework 9

### Part 1

If the weather report says it will rain, then with probability 0.8, Dana brings
her umbrella. If the weather report says it won’t rain, then with probability
0.3, Dana brings her umbrella. In Singapore, the weather report predicts rain
with probability 0.8.


#### Answer:

[**Q1**] Draw a Bayesian network corresponding to this problem, using variables U (for umbrella) and P (for predicted rain).

![part1q1](part1q1.jpg)

[**Q2**] Compute (by hand) the joint distribution over U and P

|                | P = Rain | P = No Rain |
|----------------|----------|-------------|
| U = Bring      | 0.64     | 0.06        |
| U = Dont Bring | 0.16     | 0.14        |

[**Q3**] check the code in testcase.py. Compute P(P|U = 1) and P(P|U = 0)
using the code.




In [1]:
from bayesian_network import BayesianNetwork, compute_p

single_prob = (("P", 0.8),)
CONDITION = (({"P":True}, 0.8),({"P":False}, 0.3))
conditional_prob =(("U", CONDITION),)
net = BayesianNetwork(single_prob, conditional_prob)

print("\nJoint Distributuin Check\n")

compute_p(net, event={"P":True, "U":True})
compute_p(net, event={"P":True, "U":False})
compute_p(net, event={"P":False, "U":True})
compute_p(net, event={"P":False, "U":False})

print("\nComputing Required Probabilities\n")
print("-- P(P|U=1) --\n")
compute_p(net, event={"P":True}, given={"U":True})
compute_p(net, event={"P":False}, given={"U":True})
print("\n-- P(P|U=0) --\n")
compute_p(net, event={"P":True}, given={"U":False})
compute_p(net, event={"P":False}, given={"U":False})

{'P': True}: 0.8000
{'P': False}: 0.2000
calculate children
{'P': True, 'U': True}: 0.6400
{'P': True, 'U': False}: 0.1600
{'P': False, 'U': True}: 0.0600
{'P': False, 'U': False}: 0.1400

Joint Distributuin Check

P(P,U) = 0.6400
P(P,-U) = 0.1600
P(-P,U) = 0.0600
P(-P,-U) = 0.1400

Computing Required Probabilities

-- P(P|U=1) --

P(P|U) = 0.9143
P(-P|U) = 0.0857

-- P(P|U=0) --

P(P|-U) = 0.5333
P(-P|-U) = 0.4667


[**Q4**] Really, you are not so interested in whether rain was predicted, but
whether it will actually rain. You study the prediction accuracy of weather
predictions, and decide to add a new node, R, to your network, modeling
whether it will rain today. How will the model look like as Bayesian
network?


![part1q4](part1q4.jpg)

[**Q5**] for the new network, what is the right formula for P(R = 1|P) and for P(R = 1|U)?

$$ P(R=1|P) = \frac{P(P | R=1) \cdot P(R = 1)} { P(P=1 | R=1) \cdot P(R = 1) + P(P=0 | R=1) \cdot P(R = 1)} $$

$$ P(R=1|U) = \frac{P(U | R=1) \cdot P(R = 1)} { P(U=1 | R=1) \cdot P(R = 1) + P(U=0 | R=1) \cdot P(R = 1)} $$

$$ where \quad P(U|R) = P(U|P) \cdot P(P|R)$$


[**Q6**]  What probabilities do you need? Make up any numbers that you need.
Then encode this in the Bayes net code and compute the probability of
rain given that Dana brings her umbrella or given that the report says
rain or given both observed as evidence.

#### Answer:

The following will be used:

| R = 0 | R = 1 |
|-------|-------|
| 0.75  | 0.25  |


|             | P = 0 | P = 1 |
|-------------|-------|-------|
| given R = 0 | 0.7   | 0.3   |
| given R = 1 | 0.1   | 0.9   |

Computing 
+ P(R| U=1)
+ P(R| P=1)
+ P(R| U=1, P=1)

In [2]:
single_prob = (("R", 0.75),)

P_CONDITION = (({"R":True}, 0.9),({"R":False}, 0.3))
prediction_prob = ("P", P_CONDITION)

U_CONDITION = (({"P":True}, 0.8),({"P":False}, 0.3))
umbrella_prob =("U", U_CONDITION)

conditional_prob = (prediction_prob, umbrella_prob)

net = BayesianNetwork(single_prob, conditional_prob)

print("Computing..")
compute_p(net, event={"R":True}, given={"U":True})
compute_p(net, event={"R":True}, given={"P":True})
compute_p(net, event={"R":True}, given={"U":True, "P":True})

{'R': True}: 0.7500
{'R': False}: 0.2500
calculate children
{'R': True, 'P': True, 'U': True}: 0.5400
{'R': True, 'P': True, 'U': False}: 0.1350
{'R': True, 'P': False, 'U': True}: 0.0225
{'R': True, 'P': False, 'U': False}: 0.0525
{'R': False, 'P': True, 'U': True}: 0.0600
{'R': False, 'P': True, 'U': False}: 0.0150
{'R': False, 'P': False, 'U': True}: 0.0525
{'R': False, 'P': False, 'U': False}: 0.1225
Computing..
P(R|U) = 0.8333
P(R|P) = 0.9000
P(R|U,P) = 0.9000


### Part 2

Encode your rain model from question from the updated monday lectures in the
bayesnet code. Explore explaining away by computing the following.

+ P(B = 1|A = 1)
+ P(B = 1|A = 1, R = 1) – see what observing an additional evidence, namely the radio report about earth quakes tells you about the probability of having been burgled.
+ compute both terms for P(R = 1|E = 0) = 0.0001, andP(R = 1|E = 1) =
a number which is not 1.0 as before but still high, compare what happens
as you change this number.

In [3]:
single_prob = (("B", 0.01), ("E", 1e-6),)

R_CONDITION = (({"E":True}, 1),({"E":False}, 0))
r_prob = ("R", R_CONDITION)

A_CONDITION = (({"E":True, "B":True}, 0.9999),
               ({"E":True, "B":False}, 0.99),
               ({"E":False, "B":True}, 0.99),
               ({"E":False, "B":False}, 0.0001))
a_prob = ("A", A_CONDITION)

conditional_prob = (r_prob, a_prob)

net = BayesianNetwork(single_prob, conditional_prob)

{'B': True}: 0.0100
{'B': False}: 0.9900
{'E': True}: 0.0000
{'E': False}: 1.0000
calculate children


KeyError: 'E'